In [1]:
import os
import torch
from pathlib import Path
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

from covid.model import CovidResNet18Model, CovidResNet50Model, CovidVgg19Model
from covid.module import CovidModuleClassifier
from covid.metrics import get_metrics
from covid.data import CovidDataModule
from covid.train import ModelTrainer

In [2]:
PATH_DATA = "./Covid19-dataset"

batch_size=32
learning_rate = 0.01
epochs=25
num_classes=3

## Create DataModule

In [3]:
pl_datamodule = CovidDataModule(data_path=PATH_DATA, batch_size=16)
pl_datamodule.setup("fit")
print(pl_datamodule.class_to_idx)

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}


## Calculate Class Weights

In [4]:
img_classes = os.listdir(Path(PATH_DATA, "train"))
class_count = {}
for img_class in img_classes :
    if img_class != ".DS_Store":
        class_idx = pl_datamodule.class_to_idx[img_class]
        class_count[class_idx] = len(os.listdir(path=Path(PATH_DATA, "train", img_class)))

total_samples = sum(class_count.values())
num_classes = len(class_count)

class_weights = {}
for class_idx, value_count in class_count.items():
    weight = total_samples/(num_classes*value_count)
    class_weights[class_idx] = weight
    
class_weights_tensor = torch.tensor([class_weights[i] for i in sorted(class_weights.keys())], dtype=torch.float)
print(class_weights_tensor)

tensor([0.7538, 1.1952, 1.1952])


## Transfer Learning: Resnet18 Model

In [5]:
covid_resnet18_model = CovidResNet18Model(num_classes=3, train_backbone_params=False)
metrics = get_metrics(num_classes=num_classes)
criterion = CrossEntropyLoss(weight=class_weights_tensor)
run_datadir = "./model_trained/resnet18/"


pl_model_resnet18 = CovidModuleClassifier(
    model=covid_resnet18_model,
    metrics=metrics,
    criterion=criterion
)

trainer_resnet_18 = ModelTrainer(
    pl_datamodule=pl_datamodule,
    pl_model=pl_model_resnet18,
    run_datadir=run_datadir
)

model_resnet18_trained = trainer_resnet_18.train(epochs=epochs, use_gpu=True)

/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/maicolrod

Epoch 0: 100%|██████████| 13/13 [00:06<00:00,  2.07it/s, v_num=d63c]

Metric val_loss improved. New best score: 0.772


Epoch 1: 100%|██████████| 13/13 [00:02<00:00,  6.29it/s, v_num=d63c]

Metric val_loss improved by 0.286 >= min_delta = 0.0. New best score: 0.486


Epoch 4: 100%|██████████| 13/13 [00:01<00:00,  6.71it/s, v_num=d63c]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.486. Signaling Trainer to stop.


Epoch 4: 100%|██████████| 13/13 [00:01<00:00,  6.66it/s, v_num=d63c]


In [6]:
test_metrics_resnet18 = trainer_resnet_18.test()

Restoring states from the checkpoint path at /Users/maicolrodrigues/Documents/Test/Covid/model_trained/resnet18/checkpoints/epoch=1-val_loss=0.486.ckpt
Loaded model weights from the checkpoint at /Users/maicolrodrigues/Documents/Test/Covid/model_trained/resnet18/checkpoints/epoch=1-val_loss=0.486.ckpt


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 11.40it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_macro      0.6910256743431091
   test_accuracy_micro      0.7121211886405945
 test_accuracy_weighted     0.7121212482452393
        test_loss           0.5291117429733276
  test_precision_macro      0.8013467788696289
  test_precision_micro      0.7121211886405945
 test_precision_weighted    0.8093051910400391
    test_recall_macro       0.6910256743431091
    test_recall_micro       0.7121211886405945
  test_recall_weighted      0.7121212482452393
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Transfer Learning: Resnet50 Model

In [7]:
covid_resnet50_model = CovidResNet50Model(num_classes=num_classes, train_backbone_params=False)
metrics = get_metrics(num_classes=num_classes)
criterion = CrossEntropyLoss(weight=class_weights_tensor)
run_datadir = "./model_trained/resnet50/"


pl_model_resnet50 = CovidModuleClassifier(
    model=covid_resnet50_model,
    metrics=metrics,
    criterion=criterion
)

trainer_resnet_50 = ModelTrainer(
    pl_datamodule=pl_datamodule,
    pl_model=pl_model_resnet50,
    run_datadir=run_datadir
)

model_resnet50_trained = trainer_resnet_50.train(epochs=epochs, use_gpu=True)

/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/maicolrodrigues/Documents/Test/Covid/.venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/maicolrod

Epoch 0: 100%|██████████| 13/13 [00:07<00:00,  1.77it/s, v_num=3594]

Metric val_loss improved. New best score: 0.971


Epoch 1: 100%|██████████| 13/13 [00:03<00:00,  3.48it/s, v_num=3594]

Metric val_loss improved by 0.369 >= min_delta = 0.0. New best score: 0.602


...

Epoch 11: 100%|██████████| 13/13 [00:03<00:00,  3.61it/s, v_num=3594]

Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.211


Epoch 14: 100%|██████████| 13/13 [00:03<00:00,  3.74it/s, v_num=3594]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.211. Signaling Trainer to stop.


Epoch 14: 100%|██████████| 13/13 [00:03<00:00,  3.73it/s, v_num=3594]


In [8]:
test_metrics_resnet50 = trainer_resnet_50.test()

Restoring states from the checkpoint path at /Users/maicolrodrigues/Documents/Test/Covid/model_trained/resnet50/checkpoints/epoch=11-val_loss=0.211.ckpt
Loaded model weights from the checkpoint at /Users/maicolrodrigues/Documents/Test/Covid/model_trained/resnet50/checkpoints/epoch=11-val_loss=0.211.ckpt


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00,  6.65it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_macro      0.8833333253860474
   test_accuracy_micro      0.8939393758773804
 test_accuracy_weighted     0.8939393758773804
        test_loss           0.23394981026649475
  test_precision_macro       0.883876383304596
  test_precision_micro      0.8939393758773804
 test_precision_weighted    0.8944330215454102
    test_recall_macro       0.8833333253860474
    test_recall_micro       0.8939393758773804
  test_recall_weighted      0.8939393758773804
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
